# Predict Winner from Realtime Stats using RNN

In [15]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.autograd import Variable
from tqdm import tqdm
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 0
MAX_TIME_STEP = 5

## Dataset

In [16]:
df = pd.read_csv('../data/LeagueofLegends.csv', sep=',')
df = df[df['gamelength'] >= MAX_TIME_STEP]
df.reset_index(drop = True, inplace = True)
matches = len(df)
print(f'# of matches: {matches}')

from ast import literal_eval
df['golddiff'] = df['golddiff'].apply(literal_eval)
df[['golddiff']].head()

# of matches: 7620


,golddiff
0,"[0, 0, -14, -65, -268, -431, -488, -789, -494,..."
1,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,..."
2,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ..."
3,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ..."
4,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20..."


In [17]:
def count_item(items):
    count = np.zeros(MAX_TIME_STEP, dtype=np.int8)
    for timestep in range(MAX_TIME_STEP) :
        for item in items:
            if item[0] <= timestep + 1:
                count[timestep] += 1
    return count

df['bDragons'] = df['bDragons'].apply(literal_eval)
df['rDragons'] = df['rDragons'].apply(literal_eval)

df['bDragons'] = df['bDragons'].apply(count_item)
df['rDragons'] = df['rDragons'].apply(count_item)
df['dragondiff'] = df['bDragons'] - df['rDragons']

df[['dragondiff']].head()

,dragondiff
0,"[0, 0, 0, 0, 0]"
1,"[0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0]"
3,"[0, 0, 0, 0, 0]"
4,"[0, 0, 0, 0, 0]"


In [18]:
df['bBarons'] = df['bBarons'].apply(literal_eval)
df['rBarons'] = df['rBarons'].apply(literal_eval)

df['bBarons'] = df['bBarons'].apply(count_item)
df['rBarons'] = df['rBarons'].apply(count_item)
df['barondiff'] = df['bBarons'] - df['rBarons']

df[['barondiff']].head()

,barondiff
0,"[0, 0, 0, 0, 0]"
1,"[0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0]"
3,"[0, 0, 0, 0, 0]"
4,"[0, 0, 0, 0, 0]"


In [19]:
df['bHeralds'] = df['bHeralds'].apply(literal_eval)
df['rHeralds'] = df['rHeralds'].apply(literal_eval)

df['bHeralds'] = df['bHeralds'].apply(count_item)
df['rHeralds'] = df['rHeralds'].apply(count_item)
df['heralddiff'] = df['bHeralds'] - df['rHeralds']

df[['heralddiff']].head()

,heralddiff
0,"[0, 0, 0, 0, 0]"
1,"[0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0]"
3,"[0, 0, 0, 0, 0]"
4,"[0, 0, 0, 0, 0]"


In [20]:
df['bTowers'] = df['bTowers'].apply(literal_eval)
df['rTowers'] = df['rTowers'].apply(literal_eval)

df['bTowers'] = df['bTowers'].apply(count_item)
df['rTowers'] = df['rTowers'].apply(count_item)
df['towerdiff'] = df['bTowers'] - df['rTowers']

df[['towerdiff']].head()

,towerdiff
0,"[0, 0, 0, 0, 0]"
1,"[0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0]"
3,"[0, 0, 0, 0, 0]"
4,"[0, 0, 0, 0, 0]"


In [21]:
df['bInhibs'] = df['bInhibs'].apply(literal_eval)
df['rInhibs'] = df['rInhibs'].apply(literal_eval)

df['bInhibs'] = df['bInhibs'].apply(count_item)
df['rInhibs'] = df['rInhibs'].apply(count_item)
df['inhibitordiff'] = df['bInhibs'] - df['rInhibs']

df[['inhibitordiff']].head()

,inhibitordiff
0,"[0, 0, 0, 0, 0]"
1,"[0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0]"
3,"[0, 0, 0, 0, 0]"
4,"[0, 0, 0, 0, 0]"


In [22]:
df['bKills'] = df['bKills'].apply(literal_eval)
df['rKills'] = df['rKills'].apply(literal_eval)

df['bKills'] = df['bKills'].apply(count_item)
df['rKills'] = df['rKills'].apply(count_item)
df['killdiff'] = df['bKills'] - df['rKills']

df[['killdiff']].head()

,killdiff
0,"[0, 0, 0, 0, 0]"
1,"[0, 0, 0, 0, 0]"
2,"[0, 0, 0, 0, 0]"
3,"[0, 0, 0, 0, 0]"
4,"[0, 0, 0, 0, 0]"


In [23]:
stats = ['golddiff','dragondiff', 'barondiff', 'heralddiff', 'towerdiff', 'inhibitordiff', 'killdiff']
x = df[stats]
y = df['bResult']

x.head()

,golddiff,dragondiff,barondiff,heralddiff,towerdiff,inhibitordiff,killdiff
0,"[0, 0, -14, -65, -268, -431, -488, -789, -494,...","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
1,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
2,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
3,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
4,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20...","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"


Normalize data:

In [24]:
from sklearn.preprocessing import StandardScaler

data = {}
for stat in stats:
    scaler = StandardScaler()
    for row in df[stat]:
        scaler.partial_fit(np.asanyarray(row).reshape(-1, 1))
    data[stat] = [scaler.transform(np.asanyarray(row).reshape(-1, 1)).reshape(-1) for row in df[stat]]

num_features = len(data)
print(f'# of features per timestep: {num_features}')

# of features per timestep: 7


Build dataset:

In [25]:
class LOLDataset(Dataset):
    def __init__(self, data, stats, label):
        
        self.data =[]
        for t in range(MAX_TIME_STEP):
            self.data.append([[data[stat][i][t] for stat in stats] for i in range(matches)])
        # print("shape:", np.asanyarray(self.data).shape)
        self.label=[i for i in label]
        
    
    def __getitem__(self, item):
        return torch.tensor([ [torch.scalar_tensor(i) for i in x[item]] for x in self.data]), torch.tensor(self.label[item])

    def __len__(self):
        return len(self.label)
            

## Neural Network Structure

### Recurrent Neural Network (RNN)

In [26]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN,self).__init__()
        self.hidden_size = 256
        
        self.rnn= nn.RNN(
            nonlinearity = 'relu',
            input_size = num_features,
            hidden_size = self.hidden_size,
            num_layers = 1,
            batch_first = True
        )

        self.out = nn.Linear(self.hidden_size, 2)
    
    def forward(self,x):
        r_out, hn = self.rnn(x, torch.zeros(1, len(x), self.hidden_size))
        out = self.out(r_out[:, -1, :])
        return out


#### Long Short-Term Memory (LSTM) Network

In [27]:
class LSTMRNN(nn.Module):
    def __init__(self):
        super(LSTMRNN, self).__init__()
        self.hidden_size = 256
        self.lstm = nn.LSTM(
            input_size = num_features,
            hidden_size = self.hidden_size,
            num_layers = 1,
            batch_first = True
        )
        self.out = nn.Linear(self.hidden_size, 2)
    
    def forward(self,x):
        h0 = torch.zeros(1, len(x), self.hidden_size)
        c0 = torch.zeros(1, len(x), self.hidden_size)
        r_out, (h_n, h_c) = self.lstm(x, (h0, c0))
        out = self.out(r_out[:, -1, :])
        return out

## Training

Split the dataset into Train:Held-Out:Test = 6:2:2, so we can early stop when held-out accuracy drops.

Using a batch size of 32 to load dataset for training.

In [28]:
BATCH_SIZE = 32

dataset = LOLDataset(data, stats, df["bResult"])
test_size = valid_size = int(0.2 * len(dataset))
train_size = len(dataset) - test_size - valid_size

trainDataset, validDataset, testDataset = random_split(
    dataset = dataset,
    lengths = [train_size, valid_size, test_size],
    generator = torch.Generator().manual_seed(0)
)

trainLoader = DataLoader(trainDataset, batch_size = BATCH_SIZE, shuffle=True)
validLoader = DataLoader(validDataset, batch_size = BATCH_SIZE)
testLoader = DataLoader(testDataset, batch_size = BATCH_SIZE)

In [29]:
def train(dataloader, model, loss_fn, optimizer, mute = False):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        x, y = Variable(x), Variable(y)

        predict = model(x)
        loss = loss_fn(predict, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 30 == 0 and not mute:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [30]:
def test(dataloader, model, loss_fn, validation = False):
    model.eval()
    size = len(dataloader.dataset)

    correct = 0
    test_loss = 0
    with torch.no_grad():
        for step,(x,y) in enumerate(dataloader):
            x, y = Variable(x), Variable(y)
            predict = model(x)
            # print(predict)
            test_loss += loss_fn(predict, y).item()
            correct += (predict.argmax(1) == y).sum().item()
    
    print(f"{'Valid' if validation else 'Test'} Acc:{correct/size:>7f}, Avg Loss: {test_loss/size:>7f}")
    return correct/size

### Training RNN

In [31]:
MUTE = False
EPOCH = 100
LR = 0.0001

torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

model_RNN = RNN()
print(model_RNN)

optimizer = torch.optim.Adam(model_RNN.parameters(), lr = LR)
loss_func = nn.CrossEntropyLoss()

best_acc = 0
early_stopping = 0
early_stopping_threshold = 5

for epoch in range(1, EPOCH + 1):
    print(f"--------- Epoch #{epoch} ---------")
    train(trainLoader, model_RNN, loss_func, optimizer, mute = MUTE)
    valid_acc = test(validLoader, model_RNN, loss_func, validation = True)
    if valid_acc > best_acc :
        early_stopping = 0
        best_acc = valid_acc
        torch.save(model_RNN.state_dict(), "./model/RNN_"+str(MAX_TIME_STEP)+".pt")
    else :
        early_stopping += 1
        if early_stopping == early_stopping_threshold :
            print(f"Early stopped at epoch #{epoch} with best validation accuracy {best_acc*100:.2f}%.")
            break


RNN(
  (rnn): RNN(7, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=2, bias=True)
)
--------- Epoch #1 ---------
loss: 0.684354  [    0/ 4572]
loss: 0.704525  [  960/ 4572]
loss: 0.690929  [ 1920/ 4572]
loss: 0.687046  [ 2880/ 4572]
loss: 0.687077  [ 3840/ 4572]
Valid Acc:0.608924, Avg Loss: 0.020959
--------- Epoch #2 ---------
loss: 0.672445  [    0/ 4572]
loss: 0.722603  [  960/ 4572]
loss: 0.672942  [ 1920/ 4572]
loss: 0.733238  [ 2880/ 4572]
loss: 0.735274  [ 3840/ 4572]
Valid Acc:0.607612, Avg Loss: 0.020813
--------- Epoch #3 ---------
loss: 0.660731  [    0/ 4572]
loss: 0.654829  [  960/ 4572]
loss: 0.743040  [ 1920/ 4572]
loss: 0.635377  [ 2880/ 4572]
loss: 0.657021  [ 3840/ 4572]
Valid Acc:0.612861, Avg Loss: 0.020790
--------- Epoch #4 ---------
loss: 0.679533  [    0/ 4572]
loss: 0.651014  [  960/ 4572]
loss: 0.677769  [ 1920/ 4572]
loss: 0.579066  [ 2880/ 4572]
loss: 0.644953  [ 3840/ 4572]
Valid Acc:0.610892, Avg Loss: 0.020769
--------- Epoch #5 ---

### Training LSTM RNN

In [32]:
model_LSTM = LSTMRNN()
print(model_LSTM)

LR = 0.001

optimizer = torch.optim.Adam(model_LSTM.parameters(), lr = LR)
loss_func = nn.CrossEntropyLoss()

best_acc = 0
early_stopping = 0
early_stopping_threshold = 5

for epoch in range(1, EPOCH + 1):
    print(f"--------- Epoch #{epoch} ---------")
    train(trainLoader, model_LSTM, loss_func, optimizer, mute = MUTE)
    valid_acc = test(validLoader, model_LSTM, loss_func, validation = True)
    if valid_acc > best_acc :
        early_stopping = 0
        best_acc = valid_acc
        torch.save(model_LSTM.state_dict(), "./model/LSTM_"+str(MAX_TIME_STEP)+".pt")
    else :
        early_stopping += 1
        if early_stopping == early_stopping_threshold :
            print(f"Early stopped at epoch #{epoch} with best validation accuracy {best_acc * 100:.2f}%.")
            break

LSTMRNN(
  (lstm): LSTM(7, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=2, bias=True)
)
--------- Epoch #1 ---------
loss: 0.689898  [    0/ 4572]
loss: 0.653547  [  960/ 4572]
loss: 0.644362  [ 1920/ 4572]
loss: 0.709258  [ 2880/ 4572]
loss: 0.726287  [ 3840/ 4572]
Valid Acc:0.609580, Avg Loss: 0.020890
--------- Epoch #2 ---------
loss: 0.692934  [    0/ 4572]
loss: 0.639459  [  960/ 4572]
loss: 0.694491  [ 1920/ 4572]
loss: 0.708440  [ 2880/ 4572]
loss: 0.625097  [ 3840/ 4572]
Valid Acc:0.608268, Avg Loss: 0.020696
--------- Epoch #3 ---------
loss: 0.698956  [    0/ 4572]
loss: 0.682237  [  960/ 4572]
loss: 0.633258  [ 1920/ 4572]
loss: 0.693154  [ 2880/ 4572]
loss: 0.667148  [ 3840/ 4572]
Valid Acc:0.608268, Avg Loss: 0.020759
--------- Epoch #4 ---------
loss: 0.686892  [    0/ 4572]
loss: 0.652396  [  960/ 4572]
loss: 0.682457  [ 1920/ 4572]
loss: 0.687656  [ 2880/ 4572]
loss: 0.656726  [ 3840/ 4572]
Valid Acc:0.610236, Avg Loss: 0.020708
--------- Epoch 

## Test

Load best models for both RNN and LSTM and test for results:

In [33]:
print("RNN:")
model_RNN.load_state_dict(torch.load("./model/RNN_"+str(MAX_TIME_STEP)+".pt"))
acc_RNN = test(testLoader, model_RNN, loss_func)

print("LSTM RNN:")
model_LSTM.load_state_dict(torch.load("./model/LSTM_"+str(MAX_TIME_STEP)+".pt"))
acc_LSTM = test(testLoader, model_LSTM, loss_func)

print(f'RNN Accuracy = {acc_RNN*100:>.2f}% | LSTM Accuracy = {acc_LSTM*100:>.2f}%')

RNN:
Test Acc:0.580052, Avg Loss: 0.021101
LSTM RNN:
Test Acc:0.589239, Avg Loss: 0.020900
RNN Accuracy = 58.01% | LSTM Accuracy = 58.92%


Predict S11 World Championship Final: EDG vs DK

In [35]:
match_stats = {}

''' S11 EDG vs DK, match 3, DK(red) wins '''
match_stats['golddiff'] = [0,31,16,122,-71,325,170,367,463,918,1479,1181,689,813,890,1486,2779,2215,2543,2805,2887,2975,3477,3842,3361,3450,3318,3101,754,752]
match_stats['dragondiff'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -2, -2, -2, -2, -2, -2, -3, -3, -3, -3, -3, -3, -4, -4]
match_stats['barondiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
match_stats['heralddiff'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
match_stats['towerdiff'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2]
match_stats['inhibitordiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
match_stats['killdiff'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -1, -1]

''' S11 EDG vs DK, match 4, EDG(blue) wins '''
# match_stats['golddiff'] = [0,-33,147,160,-124,-627,621,979,577,759,1136,1175,-39,554,2057,1839,2071,1771,2058,2515,2297,1408,1575,1459,1521,1525,2672,2314,2717,2417,]
# match_stats['dragondiff'] = [0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,4,4,4,4]
# match_stats['barondiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# match_stats['heralddiff'] = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
# match_stats['towerdiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1]
# match_stats['inhibitordiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# match_stats['killdiff'] = [0,0,0,0,0,0,1,1,1,1,1,1,-1,-1,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1]


''' S11 DK vs EDG, match 5, EDG(red) wins ''' 
# match_stats['golddiff'] = [0,-21,-108,135,125,451,233,133,-431,-810,-860,-1128,-794,-774,-797,-886,-401,-216,-321,35,-143,-55,-734,-1719,-1955,-1864,-2100,-487,-324,-3383]
# match_stats['dragondiff'] = [0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,-1,-1,-1,-1,-1,-2,-2,-2,-2,-2,-2,-3]
# match_stats['barondiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1]
# match_stats['heralddiff'] = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
# match_stats['towerdiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,-1]
# match_stats['inhibitordiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# match_stats['killdiff'] = [0,0,0,0,0,0,0,0,-1,-1,-2,-2,-3,-3,-3,-2,-2,-2,-2,-2,-2,-2,-5,-6,-6,-6,-6,-6,-6,-10]

for stat in stats:
    match_stats[stat] = scalers[stat].transform(np.asanyarray(match_stats[stat]).reshape(-1, 1)).reshape(-1)

x = np.asarray([[ [match_stats[stat][timestep] for stat in stats] for timestep in range(MAX_TIME_STEP) ]], dtype=np.float32)

print(x.shape)

def predict_one_match(model, x):
    model.eval()
    with torch.no_grad():
        x = torch.from_numpy(x)
        predict = model(x)
        # print(predict)
        winner = ['red', 'blue'][predict.argmax(1)]
        from math import exp
        prob_red = exp(predict[0][0].item()) / (exp(predict[0][0].item()) + exp(predict[0][1].item()))
        prob_blue = exp(predict[0][1].item()) / (exp(predict[0][0].item()) + exp(predict[0][1].item()))
        print(f"model predicted winner: { winner }")
        print(f"red wins: {prob_red * 100 :.1f}% | blue wins: {prob_blue * 100:.1f}%")

print("RNN:")
predict_one_match(model_RNN, x)
print("LSTM RNN:")
predict_one_match(model_LSTM, x)

(1, 5, 7)
RNN:
model predicted winner: blue
red wins: 45.9% | blue wins: 54.1%
LSTM RNN:
model predicted winner: blue
red wins: 46.6% | blue wins: 53.4%
